In [1]:
import pandas as pd
import torch
import skimage
import pickle

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm

In [2]:
from utils import *

In [3]:
BASE_DIR = "/vol/bitbucket/pn222/satellite/dataloader/64_128/"

In [4]:
import sys
sys.stdout = open(f'DL_LOG_{datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")}.log','wt')

In [5]:
cyclones_path = "./list_of_cyclones.xlsx"
df = pd.read_excel(cyclones_path)
df = df.drop('Unnamed: 8', axis=1)
df = df.dropna()

In [6]:
def is_stub_already_present(dest_folder, stub):
    stubs = [x.split('/')[-1] for x in glob.glob(dest_folder+"*.dat")]
    if stub in stubs: 
        print(f"Present: {stub}", flush=True)
        return True
    return False

In [7]:
def fetch_cyclone(idx):
    row = df.iloc[idx]
    region = row["Region"]
    name = row["Name"]
    cyclone = Cyclone(region, name)
    cyclone.load_era5()
    
    o_size = 64 ; n_size = 128
    data_loader = CycloneDataLoader()

    region = region_to_abbv[region]
    name = name.replace(' ', '').lower()
    filename = f"{region}_{name}.dat"
    
    if is_stub_already_present(BASE_DIR, filename):
        return

    print(f"[{name.upper()}] Processing dataloader.", flush=True)

    for satmap_idx in tqdm(range(cyclone.metadata['count'])):
        ir108_fn = cyclone.metadata['satmaps'][satmap_idx]['ir108_fn']
        ir108_scn = cyclone.get_ir108_data(ir108_fn)    
        img = ir108_scn.to_numpy() ; 
        img = transform_make_sq_image(img)
    
        img_o = skimage.transform.resize(img, (o_size, o_size), anti_aliasing=True)
        img_o = torch.from_numpy(img_o).unsqueeze(0)        
    
        img_n = skimage.transform.resize(img, (n_size, n_size), anti_aliasing=True)
        img_n = torch.from_numpy(img_n).unsqueeze(0)
        
        era5_idx = cyclone.metadata['satmaps'][satmap_idx]['era5_idx']
        era5 = cyclone.get_era5_data(era5_idx, gfs=True)
        era5 = skimage.transform.resize(era5, (3, n_size, n_size), anti_aliasing=True)
        era5 = torch.from_numpy(era5).unsqueeze(0)

        if torch.isnan(img_o.sum()) or torch.isnan(img_n.sum()) or torch.isnan(era5.sum()):
            print(f"[NAN]\t{region}\t{name}\t{satmap_idx}", flush=True)
            continue
        
        data_loader.add_image(img_o, img_n, era5)
    
    with open(f'{BASE_DIR}{filename}', 'wb') as data_file:
        pickle.dump(data_loader, data_file)

    print(f"[{name.upper()}] Completed processing dataloader.", flush=True)

In [8]:
%%time

from multiprocessing import Pool, cpu_count
from functools import partial

idx = list(range(5))

pool = Pool(cpu_count())
fetch_cyclone_func = partial(fetch_cyclone)
results = pool.map(fetch_cyclone_func, idx)
pool.close()
pool.join()

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 158/158 [04:33<00:00,  1.73s/it]
